In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('sales.csv')

In [3]:
df.head()

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales
0,0,625,3,2013-11-06,641,1,1,0,0,7293
1,1,293,2,2013-07-16,877,1,1,0,1,7060
2,2,39,4,2014-01-23,561,1,1,0,0,4565
3,3,676,4,2013-09-26,1584,1,1,0,0,6380
4,4,709,3,2014-01-22,1477,1,1,0,0,11647


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640840 entries, 0 to 640839
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   True_index           640840 non-null  int64 
 1   Store_ID             640840 non-null  int64 
 2   Day_of_week          640840 non-null  int64 
 3   Date                 640840 non-null  object
 4   Nb_customers_on_day  640840 non-null  int64 
 5   Open                 640840 non-null  int64 
 6   Promotion            640840 non-null  int64 
 7   State_holiday        640840 non-null  object
 8   School_holiday       640840 non-null  int64 
 9   Sales                640840 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 48.9+ MB


In [5]:

# Eliminar la columna 'True_index'
df.drop('True_index', axis=1, inplace=True)

# Convertir la columna 'Date' a formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Calcular la fecha máxima en la columna 'Date'
max_date = df['Date'].max()

# Calcular la diferencia entre la fecha máxima y cada fecha
df['Days_from_max_date'] = (max_date - df['Date']).dt.days

# Extraer el día y el mes de la columna 'Date'
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month

# Convertir el día y el mes a coseno (normalizado para 360 grados, como ciclos de año)
df['Day_cos'] = np.cos(2 * np.pi * df['Day'] / 31)  # Dividimos entre 31 para normalizar días del mes
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)  # Dividimos entre 12 para normalizar los meses

# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(df, columns=['Open', 'Promotion', 'State_holiday', 'School_holiday'], drop_first=True)

# Eliminar la columna 'Date'
df.drop('Date', axis=1, inplace=True)

In [6]:

df.head()

,Store_ID,Day_of_week,Nb_customers_on_day,Sales,Days_from_max_date,Day,Month,Day_cos,Month_cos,Open_1,Promotion_1,State_holiday_a,State_holiday_b,State_holiday_c,School_holiday_1
0,625,3,641,7293,632,6,11,0.347305,8.660254e-01,True,True,False,False,False,False
1,293,2,877,7060,745,16,7,-0.994869,-8.660254e-01,True,True,False,False,False,True
2,39,4,561,4565,554,23,1,-0.050649,8.660254e-01,True,True,False,False,False,False
3,676,4,1584,6380,673,26,9,0.528964,-1.836970e-16,True,True,False,False,False,False
4,709,3,1477,11647,555,22,1,-0.250653,8.660254e-01,True,True,False,False,False,False


In [7]:
X = df.drop('Sales', axis=1)  # Features: todas las columnas excepto 'Sales'
y = df['Sales']  # Target: la columna 'Sales'

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:

# Entrenar el modelo Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)

# Entrenar el modelo Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(n_estimators=50, random_state=42)
gb_model.fit(X_train, y_train)

# Mostrar los resultados de ambos modelos entrenados
print("Modelos entrenados con éxito:")
print("- Random Forest Regressor")
print("- Gradient Boosting Regressor")

Modelos entrenados con éxito:
- Random Forest Regressor
- Gradient Boosting Regressor


In [24]:
y_pred_rf = rf_model.predict(X_test)
# Calcular el Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred_rf)

# Calcular el Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_rf)

# Calcular el R^2 score
r2 = r2_score(y_test, y_pred_rf)

# Mostrar las métricas de evaluación
print("Métricas de evaluación del modelo Random Forest:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R^2 Score: {r2:.2f}")

Métricas de evaluación del modelo Random Forest:
Mean Squared Error (MSE): 929815.89
Mean Absolute Error (MAE): 613.07
R^2 Score: 0.94


In [25]:
y_pred_gb = gb_model.predict(X_test)

# Calcular el Mean Squared Error (MSE)
mse_gb = mean_squared_error(y_test, y_pred_gb)

# Calcular el Mean Absolute Error (MAE)
mae_gb = mean_absolute_error(y_test, y_pred_gb)

# Calcular el R^2 score
r2 = r2_score(y_test, y_pred_gb)

# Mostrar las métricas de evaluación
print("Métricas de evaluación del modelo Random Forest:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R^2 Score: {r2:.2f}")


Métricas de evaluación del modelo Random Forest:
Mean Squared Error (MSE): 929815.89
Mean Absolute Error (MAE): 613.07
R^2 Score: 0.88


In [26]:
df2 = pd.read_csv("ironkaggle_notarget.csv")

In [27]:
df2.head()

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday
0,7,764,4,2013-12-26,0,0,0,c,1
1,19,22,3,2013-05-22,449,1,0,0,1
2,31,1087,6,2013-06-29,622,1,0,0,0
3,45,139,6,2013-08-17,314,1,0,0,0
4,56,568,1,2014-04-07,356,1,0,0,0


In [28]:
# Suponemos que ya tienes un DataFrame llamado df2
# Eliminar la columna 'True_index'
df2.drop('True_index', axis=1, inplace=True)

# Convertir la columna 'Date' a formato datetime
df2['Date'] = pd.to_datetime(df2['Date'])

# Calcular la fecha máxima en la columna 'Date'
max_date_df2 = df2['Date'].max()

# Calcular la diferencia entre la fecha máxima y cada fecha
df2['Days_from_max_date'] = (max_date_df2 - df2['Date']).dt.days

# Extraer el día y el mes de la columna 'Date'
df2['Day'] = df2['Date'].dt.day
df2['Month'] = df2['Date'].dt.month

# Convertir el día y el mes a coseno (normalizado para 360 grados, como ciclos de año)
df2['Day_cos'] = np.cos(2 * np.pi * df2['Day'] / 31)  # Normalización de días del mes
df2['Month_cos'] = np.cos(2 * np.pi * df2['Month'] / 12)  # Normalización de los meses

# Convertir las variables categóricas en variables dummy
df2 = pd.get_dummies(df2, columns=['Open', 'Promotion', 'State_holiday', 'School_holiday'], drop_first=True)

# Eliminar la columna 'Date'
df2.drop('Date', axis=1, inplace=True)

In [29]:
df2.head()

,Store_ID,Day_of_week,Nb_customers_on_day,Days_from_max_date,Day,Month,Day_cos,Month_cos,Open_1,Promotion_1,State_holiday_a,State_holiday_b,State_holiday_c,School_holiday_1
0,764,4,0,582,26,12,0.528964,1.000000,False,False,False,False,True,True
1,22,3,449,800,22,5,-0.250653,-0.866025,True,False,False,False,False,True
2,1087,6,622,762,29,6,0.918958,-1.000000,True,False,False,False,False,False
3,139,6,314,713,17,8,-0.954139,-0.500000,True,False,False,False,False,False
4,568,1,356,480,7,4,0.151428,-0.500000,True,False,False,False,False,False


In [ ]:
#SE USA EL MODELO ENTRENADO CON Random Forest Regressor COMO EL MEJOR MODELO

In [30]:
y_pred_rf = rf_model.predict(df2)
print("Predicciones de ventas:")
print(y_pred_rf)

Predicciones de ventas:
[    0.    3703.26  6506.4  ... 15939.86     0.    4061.  ]


In [31]:
# Agregar las predicciones como una nueva columna en df2
df2['Predicted_Sales'] = y_pred_rf

# Mostrar las primeras filas del DataFrame con las predicciones
print(df2[['Store_ID', 'Day_of_week', 'Nb_customers_on_day', 'Predicted_Sales']].head())

   Store_ID  Day_of_week  Nb_customers_on_day  Predicted_Sales
0       764            4                    0             0.00
1        22            3                  449          3703.26
2      1087            6                  622          6506.40
3       139            6                  314          3642.84
4       568            1                  356          3650.18
